In [1]:
import functools
import math
import os
import sys
import time

# import holoviews as hv
# from holoviews import opts
# from holoviews.operation import contours
# hv.extension('matplotlib')
# hv.extension('bokeh')
from glob import glob

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pandas import HDFStore

sns.set_theme()
sys.path.insert(0, "..")
import scipy.stats as stats
from scipy.optimize import curve_fit
from scipy.stats import kurtosis, norm

In [2]:
files = glob("./VEP/*")

In [3]:
def calc_del(df):
    if [col for col in df] != ["dis", "ang", "VE"]:
        print('The function assumes data with "dis", "ang", and "VE".\n')
        return None
    arr = df.to_numpy()
    arr = np.pad(arr, ((0, 0), (0, 2)), mode="constant")
    for v in np.unique(arr[:, 0]):  # For each distance
        diff = (
            np.roll(arr[arr[:, 0] == v], shift=-1, axis=0) - arr[arr[:, 0] == v]
        )  # Take difference (next ang-ang)
        diff[diff[:, 1] < 0, 1] = (
            diff[diff[:, 1] < 0, 1] + 360
        )  # If angle change is negative (i.e around 0) add 360
        diff[:, 3] = (
            360 * diff[:, 2] / diff[:, 1]
        )  # angles (diff[L,1]) should be in radians
        arr[arr[:, 0] == v, 3] = diff[:, 3]

    for v in np.unique(arr[:, 1]):  # For each angle
        diff = (
            np.roll(arr[arr[:, 1] == v], shift=-1, axis=0) - arr[arr[:, 1] == v]
        )  # Take difference (next dis-dis)
        diff[:, 4] = diff[:, 2] / diff[:, 0]
        diff[
            diff[:, 0] < 0, 4
        ] = 0  # If distance change is negative (i.e around max_dis) make 0
        arr[arr[:, 1] == v, 4] = diff[:, 4]

    df = pd.DataFrame(data=arr, columns=["dis", "ang", "VE", "delr", "delo"])
    return df

In [4]:
for file in files[12:13]:
    VEP = pd.read_hdf(file, "table")
    VEP["ang"] = VEP["ang"].astype(np.float64)
    df = calc_del(VEP)
    df["tetha"] = np.radians(df["ang"])
    df["delx"] = (
        np.cos(df["tetha"]) * df["delr"]
        - 1 / df["dis"] * np.sin(df["tetha"]) * df["delo"]
    )
    df["dely"] = (
        np.sin(df["tetha"]) * df["delr"]
        + 1 / df["dis"] * np.cos(df["tetha"]) * df["delo"]
    )
    print(df)

        dis        ang     VE          delr         delo     tetha  \
0      0.36    0.10986   4.70    -97.673686    -9.333333  0.001917   
1      0.39    0.10986   4.42    -40.959933     5.000000  0.001917   
2      0.42    0.10986   4.57   -283.568765    -1.000000  0.001917   
3      0.45    0.10986   4.54    286.719529     5.666667  0.001917   
4      0.48    0.10986   4.71    132.332091    36.000000  0.001917   
...     ...        ...    ...           ...          ...       ...   
32230  3.36  358.94531   7.81   -108.196299   722.333333  6.264777   
32231  3.39  358.94531  29.48  -8380.576188 -2270.666667  6.264777   
32232  3.42  358.94531 -38.64  16291.271307  2584.666667  6.264777   
32233  3.45  358.94531  38.90  -9610.922674 -1091.666667  6.264777   
32234  3.48  358.94531   6.15   -661.543085     0.000000  6.264777   

               delx        dely  
0        -97.623795  -26.113160  
1        -40.984440   12.741952  
2       -283.563679   -2.924668  
3        286.694857   1